# Pyquil Tutorials

We have made a binder that converts QLM circuits into pyquil circuits, we have also made the `PyquilQPU` class which is a qpu wrapper around pyquil's simulator/real chip which takes QLM jobs, converts them to pyquil jobs, executes them on a pyquil simulator/real quantum chip, then returns a QLM Result.

## QLM to Pyquil conversion

A function that converts a QLM circuit into a pyquil circuit

In [1]:
from qat.interop.pyquil.converters import to_pyquil_circ
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *

nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

# Conversion
pyquil_circuit = to_pyquil_circ(qlm_circuit)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Use importlib.util.find_spec() instead.
  app.launch_new_instance()


## PyquilQPU

The wrapper around whatever pyquil simulator or quantum chip you wish to use.
In this example we use the [qvm](https://github.com/rigetti/qvm) we have running as server. We used this same method to show the qiskit connector working with pyquilqpu to execute qiskit code on pyquil's qvm and retrieving the result in QLM format.

In [1]:
from qat.interop.pyquil.providers import PyquilQPU
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *

from pyquil.api import QVMConnection

nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

qlm_job = qlm_circuit.to_job(nbshots=1024)

qvm = QVMConnection(endpoint='http://127.0.0.1:15011')

qpu = PyquilQPU(qvm)

result = qpu.submit(qlm_job)
for entry in result.raw_data:
    print("State: {}\t probability: {}".format(entry.state, entry.probability))


State: |11>	 probability: 1.0


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Use importlib.util.find_spec() instead.
